Data structures in python
* String
* List
* Tuples
* Dictionaries
* Set

*args

**kwargs

In [41]:
# *args -> positional arguments

def get_prod(*args):
    res = 1
    for arg in args:
        res = res*arg
    return res

get_prod(1, 4, 5, 2, 3, 6)

720

In [42]:
# **kwargs

def greet(**kwargs):
    greeting = "Hello"
    if "name" in kwargs:
        greeting += f", {kwargs['name']}"
    if 'age' in  kwargs:
        greeting += f", You are {kwargs['age']} years old"
    if "location" in kwargs:
        greeting += f" from {kwargs['location']}"
    greeting+="!"
    return greeting

print(greet(name='John'))
print(greet(name='John', age=24))
print(greet(name="John", location="New york"))
print(greet(name="John", age=24, location="New york"))

Hello, John!
Hello, John, You are 24 years old!
Hello, John from New york!
Hello, John, You are 24 years old from New york!


In [43]:
def check_args(*args):
    return args

def check_kwargs(**kwargs):
    return kwargs

In [44]:
check_args(1, 2, 3, 4)

(1, 2, 3, 4)

In [45]:
check_kwargs(name="John", age=24, location="New york")

{'name': 'John', 'age': 24, 'location': 'New york'}

In [46]:
# Decorators

import time
from datetime import datetime

get_curr_time = lambda : datetime.now().time().strftime("%H:%M:%S")

def timer(func):
    def get_timings(*args, **kwargs):
        start_time = get_curr_time()
        val = func(*args, **kwargs)
        time.sleep(2)
        end_time = get_curr_time()
        print(f"Start time: {start_time} | End time: {end_time} ")
        return val
    return get_timings

@timer
def get_sq(a):
    return a**2

In [25]:
get_sq(4)

Start time: 10:32:53 | End time: 10:32:55 


16

# Chains

In [5]:
import os
from dotenv import load_dotenv
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate

load_dotenv()
llm = OpenAI()
chat = ChatOpenAI()
set_llm_cache(InMemoryCache())

## LLMChain

In [6]:
human_template = "Write a film story outline on the topic: {topic}"

human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

In [7]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat, prompt=chat_prompt)

In [8]:
result = chain.invoke(input={"topic": "Growth of India"})
result

{'topic': 'Growth of India',
 'text': 'Title: Rising from the Ashes: The Growth of India\n\nAct 1:\n- The film opens with a montage of historical footage showcasing the struggles and challenges faced by India in the past, including colonization, poverty, and political instability.\n- We are introduced to our protagonist, Ravi, a young man from a small village in India who dreams of creating a better future for himself and his country.\n- Ravi moves to the bustling city of Mumbai to pursue his education and career, facing numerous obstacles along the way.\n\nAct 2:\n- As Ravi works his way up the corporate ladder, he witnesses firsthand the rapid economic growth and development taking place in India.\n- Through his experiences, Ravi learns about the importance of innovation, entrepreneurship, and hard work in driving the country\'s progress.\n- Ravi becomes involved in various initiatives aimed at promoting education, healthcare, and sustainable development in India, inspiring others to

In [9]:
chat_prompt.format(**{"topic": "Growth of India"})

'Human: Write a film story outline on the topic: Growth of India'

In [10]:
print(result['text'])

Title: Rising from the Ashes: The Growth of India

Act 1:
- The film opens with a montage of historical footage showcasing the struggles and challenges faced by India in the past, including colonization, poverty, and political instability.
- We are introduced to our protagonist, Ravi, a young man from a small village in India who dreams of creating a better future for himself and his country.
- Ravi moves to the bustling city of Mumbai to pursue his education and career, facing numerous obstacles along the way.

Act 2:
- As Ravi works his way up the corporate ladder, he witnesses firsthand the rapid economic growth and development taking place in India.
- Through his experiences, Ravi learns about the importance of innovation, entrepreneurship, and hard work in driving the country's progress.
- Ravi becomes involved in various initiatives aimed at promoting education, healthcare, and sustainable development in India, inspiring others to do the same.

Act 3:
- Just as things seem to be 

In [11]:
lcel_chain = chat_prompt | chat

lcel_chain.invoke(input={"topic": "Growth of India"})

AIMessage(content='Title: Rising from the Ashes: The Growth of India\n\nAct 1:\n- The film opens with a montage of historical footage showcasing the struggles and challenges faced by India in the past, including colonization, poverty, and political instability.\n- We are introduced to our protagonist, Ravi, a young man from a small village in India who dreams of creating a better future for himself and his country.\n- Ravi moves to the bustling city of Mumbai to pursue his education and career, facing numerous obstacles along the way.\n\nAct 2:\n- As Ravi works his way up the corporate ladder, he witnesses firsthand the rapid economic growth and development taking place in India.\n- Through his experiences, Ravi learns about the importance of innovation, entrepreneurship, and hard work in driving the country\'s progress.\n- Ravi becomes involved in various initiatives aimed at promoting education, healthcare, and sustainable development in India, inspiring others to do the same.\n\nAct

# Simple Sequential Chain

In [12]:
story_line_template = "Write a film story outline on the topic: {topic}"
story_line_prompt = ChatPromptTemplate.from_template(template=story_line_template)
story_line_chain = LLMChain(llm=chat, prompt=story_line_prompt)

full_story_template = "Write a short film story on the given story line: {story_line}"
full_story_prompt = ChatPromptTemplate.from_template(template=full_story_template)
full_story_chain = LLMChain(llm=chat, prompt=full_story_prompt)

reviewer_template = "Act as a reviewer of rotten tomatoes and rate the given story: {full_story}"
reviewer_prompt = ChatPromptTemplate.from_template(template=reviewer_template)
reviewer_chain = LLMChain(llm=chat, prompt=reviewer_prompt)

In [13]:
from langchain.chains import SimpleSequentialChain

film_chain = SimpleSequentialChain(
    chains=[story_line_chain, full_story_chain, reviewer_chain],
    verbose=True
)

In [14]:
result = film_chain.run("Growth of India")

d:\CodeWork\GitHub\langchain_training\.conda\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
Title: Rising from the Ashes: The Growth of India

Act 1:
- The film opens with a montage of historical footage showcasing the struggles and challenges faced by India in the past, including colonization, poverty, and political instability.
- We are introduced to our protagonist, Ravi, a young man from a small village in India who dreams of creating a better future for himself and his country.
- Ravi moves to the bustling city of Mumbai to pursue his education and career, facing numerous obstacles along the way.

Act 2:
- As Ravi works his way up the corporate ladder, he witnesses firsthand the rapid economic growth and development taking place in India.
- Through his experiences, Ravi learns about the importance of innovation, entrepreneurship, and hard work in driving the country's progress.
- Ravi becomes involved in various initiatives aimed at promoting education, healthcare, and sustainable development in India, inspiring others to d

Fade in to a beautiful sunrise over the city of Mumbai, showcasing the bustling streets and towering skyscrapers. Ravi stands at the edge of a rooftop, looking out at the city he now calls home. He takes a deep breath, feeling a sense of pride and accomplishment for all that he has achieved.

As Ravi looks out at the horizon, he is reminded of the journey he has been on - the struggles, the setbacks, and the triumphs. He thinks back to his humble beginnings in a small village, where he dared to dream of a better future for himself and his country.

Ravi's phone buzzes, interrupting his thoughts. He answers the call, a smile spreading across his face as he hears the voice on the other end. It's his friends and colleagues, ready to embark on their next mission to drive positive change in India.

With renewed determination, Ravi joins his allies as they set out to tackle the challenges that lie ahead. Together, they work tirelessly to promote education, healthcare, and sustainable develop

In [15]:
result

'Rating: 4/5\n\n"Rising from the Ashes: The Growth of India" is a heartwarming and inspirational story that captures the spirit of determination and resilience of the Indian people. The film beautifully portrays the journey of Ravi and his allies as they work towards creating a better future for their country. The characters are well-developed and the storyline is engaging, keeping the audience invested from start to finish.\n\nThe cinematography of the film is stunning, with breathtaking shots of Mumbai and the city\'s vibrant energy. The themes of education, healthcare, and sustainable development are important and relevant, adding depth to the narrative.\n\nWhile the plot may be a bit predictable at times, the overall message of hope and optimism shines through. The film does a great job of highlighting the challenges and obstacles that come with driving positive change, while also emphasizing the power of unity and perseverance.\n\nOverall, "Rising from the Ashes: The Growth of Ind

In [16]:
lcel_film_chain = story_line_chain | full_story_chain | reviewer_chain

lcel_film_chain.invoke(input={"topic": "Growth of India", "story_line": "Growth of India", "full_story": "Growth of India"})

{'topic': 'Growth of India',
 'story_line': 'Growth of India',
 'full_story': 'Growth of India',
 'text': 'Title: Growth of India\n\nRating: 80%\n\nReview:\n"Growth of India is a compelling and informative documentary that explores the economic and social development of India over the past few decades. The film delves into various aspects of India\'s growth story, from its booming IT sector to its expanding middle class. The interviews with economists, politicians, and ordinary citizens provide a well-rounded perspective on the challenges and opportunities facing the country. The cinematography is stunning, capturing the vibrant colors and diverse landscapes of India. However, at times, the film feels a bit too focused on the positives and fails to address some of the darker aspects of India\'s growth, such as income inequality and environmental degradation. Overall, Growth of India is a must-watch for anyone interested in understanding the complexities of India\'s economic transformat

In [18]:
from langchain.schema.runnable import RunnableLambda

def get_story_line(response):
    return {"story_line": response['text']}

def get_full_story(response):
    return {"full_story": response['text']}

story_line_lcel_chain = story_line_chain | RunnableLambda(get_story_line)
full_story_lcel_chain = full_story_chain | RunnableLambda(get_full_story)

lcel_film_chain = story_line_lcel_chain | full_story_lcel_chain | reviewer_chain

lcel_film_chain.invoke(input={"topic": "Growth of India"})

{'full_story': 'Fade in to a beautiful sunrise over the city of Mumbai, showcasing the bustling streets and towering skyscrapers. Ravi stands at the edge of a rooftop, looking out at the city he now calls home. He takes a deep breath, feeling a sense of pride and accomplishment for all that he has achieved.\n\nAs Ravi looks out at the horizon, he is reminded of the journey he has been on - the struggles, the setbacks, and the triumphs. He thinks back to his humble beginnings in a small village, where he dared to dream of a better future for himself and his country.\n\nRavi\'s phone buzzes, interrupting his thoughts. He answers the call, a smile spreading across his face as he hears the voice on the other end. It\'s his friends and colleagues, ready to embark on their next mission to drive positive change in India.\n\nWith renewed determination, Ravi joins his allies as they set out to tackle the challenges that lie ahead. Together, they work tirelessly to promote education, healthcare,

# Sequential Chain

In [26]:
from langchain.chains import SequentialChain

In [29]:
story_line_template = "Write a film story outline on the topic: {topic}"
story_line_prompt = ChatPromptTemplate.from_template(template=story_line_template)
story_line_chain = LLMChain(llm=chat, prompt=story_line_prompt, output_key="story_line")

full_story_template = "Write a short film story on the given story line: {story_line}"
full_story_prompt = ChatPromptTemplate.from_template(template=full_story_template)
full_story_chain = LLMChain(llm=chat, prompt=full_story_prompt, output_key="full_story")

reviewer_template = "Act as a reviewer of rotten tomatoes and rate the given story: {full_story}\n and the topic is: {topic}"
reviewer_prompt = ChatPromptTemplate.from_template(template=reviewer_template)
reviewer_chain = LLMChain(llm=chat, prompt=reviewer_prompt, output_key="reviwer_response")

In [30]:
lcel_chain = story_line_chain | full_story_chain | reviewer_chain

In [31]:
lcel_chain.invoke(input={"topic": "Growth of India"})

{'topic': 'Growth of India',
 'story_line': 'Title: Rising from the Ashes: The Growth of India\n\nAct 1:\n- The film opens with a montage of historical footage showcasing the struggles and challenges faced by India in the past, including colonization, poverty, and political instability.\n- We are introduced to our protagonist, Ravi, a young man from a small village in India who dreams of creating a better future for himself and his country.\n- Ravi moves to the bustling city of Mumbai to pursue his education and career, facing numerous obstacles along the way.\n\nAct 2:\n- As Ravi works his way up the corporate ladder, he witnesses firsthand the rapid economic growth and development taking place in India.\n- Through his experiences, Ravi learns about the importance of innovation, entrepreneurship, and hard work in driving the country\'s progress.\n- Ravi becomes involved in various initiatives aimed at promoting education, healthcare, and sustainable development in India, inspiring oth

In [32]:
seq_chain = SequentialChain(
    chains=[story_line_chain, full_story_chain, reviewer_chain],
    input_variables=['topic'],
    output_variables = ['story_line', 'full_story', 'reviwer_response'],
    verbose=True
)

seq_chain.invoke(input={"topic": "Growth of India"})



> Entering new SequentialChain chain...

> Finished chain.


{'topic': 'Growth of India',
 'story_line': 'Title: Rising from the Ashes: The Growth of India\n\nAct 1:\n- The film opens with a montage of historical footage showcasing the struggles and challenges faced by India in the past, including colonization, poverty, and political instability.\n- We are introduced to our protagonist, Ravi, a young man from a small village in India who dreams of creating a better future for himself and his country.\n- Ravi moves to the bustling city of Mumbai to pursue his education and career, facing numerous obstacles along the way.\n\nAct 2:\n- As Ravi works his way up the corporate ladder, he witnesses firsthand the rapid economic growth and development taking place in India.\n- Through his experiences, Ravi learns about the importance of innovation, entrepreneurship, and hard work in driving the country\'s progress.\n- Ravi becomes involved in various initiatives aimed at promoting education, healthcare, and sustainable development in India, inspiring oth

# LCEL

In [33]:
from langchain.schema.runnable import RunnableLambda

def square(a):
    return a**2

sq_runnable = RunnableLambda(square)

sq_runnable

RunnableLambda(square)

In [34]:
type(sq_runnable)

langchain_core.runnables.base.RunnableLambda

In [35]:
sq_runnable.invoke(4)

16

In [38]:
from langchain.schema.runnable import RunnablePassthrough

runnable_sq_pass = RunnablePassthrough()

In [40]:
runnable_sq_pass.invoke(5)

5

In [56]:
from langchain.schema.runnable import RunnableParallel
from operator import itemgetter


def times2(x):
    return x*2


def times3(y):
    return y*3


# def add(res_dict):
#     return res_dict['a'] + res_dict['b']
def add(res_dict):
    res = 0
    for k, v in res_dict.items():
        res+=v
    return res

runnable_times2 = RunnableLambda(times2)
runnable_times3 = RunnableLambda(times3)
runnable_sum = RunnableLambda(add)

# par_chain = RunnableParallel({"a": itemgetter('x') | runnable_times2, 
#                               "b": itemgetter('y') | runnable_times3})

par_chain = RunnableParallel(
    {"a": itemgetter('x') | runnable_times2, 
     "b": itemgetter('y') | runnable_times3,
     "x": itemgetter('x') | RunnablePassthrough(),
     "y": itemgetter('y') | RunnablePassthrough(),
     })

calc_chain = par_chain | runnable_sum

calc_chain.invoke(input={"x": 2, "y": 3})

18

In [67]:
class RunnableLambdaTest:
    def __init__(self, func):
        self.func = func
        
    def __or__(self, other_runnable_obj):
        def chained_func(*args, **kwargs):
            return other_runnable_obj.invoke(self.func(*args, **kwargs))
        return RunnableLambdaTest(chained_func)
    
    def invoke(self, *args, **kwargs):
        return self.func(*args, **kwargs)
    
def times2(a):
    return a*2

def times3(b): 
    return b*3

runnable_2 = RunnableLambdaTest(times2)
runnable_3 = RunnableLambdaTest(times3)

# test_chain = runnable_2.__or__(runnable_3)

test_chain = runnable_2 | runnable_3

test_chain.invoke(2)

12

# LLMRouterChain

In [17]:
import os
import warnings
from dotenv import load_dotenv
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

warnings.filterwarnings("ignore")
load_dotenv()
llm = OpenAI()
chat = ChatOpenAI()
set_llm_cache(InMemoryCache())

In [2]:
# Task types

psy_name = "Psychologist" # Who

psy_description = "Helps with any psychological issues" # What

psy_template = """You are a helpful psychologist who wants to help the user to make them feel better.
You always give 3 simple and most appropriate remedies to the user based on their issue.
The user issue is mentioned in the below triple backticks:
```
{input}
```
""" # How

# ----------------------------------------------

phy_name = "General Physician"

phy_description = "Helps with any General Physical Health issues"

phy_template = """You are a helpful general physician who always suggests home remedies for any issues.
You always suggest 5 most appropriate remedies to the user based on their issue.
The user issue is mentioned in the below triple backticks:
```
{input}
```
"""

In [3]:
llm_dict = {psy_name: chat, phy_name: chat}

In [4]:
prompt_info_list = [
    {
        "name": psy_name,
        "description": psy_description,
        "template": psy_template
    }, 
    {
        "name": phy_name,
        "description": phy_description,
        "template": phy_template
    }, 
]

In [8]:
destination_chains = {}
destination_descriptions = ""

for prompt_info in prompt_info_list:
    name = prompt_info['name']
    template = prompt_info['template']
    prompt = ChatPromptTemplate.from_template(template=template)
    llm = llm_dict[name]
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
    
    destination_descriptions += f"{prompt_info['name']}: {prompt_info['description']}\n"

In [9]:
destination_chains

{'Psychologist': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a helpful psychologist who wants to help the user to make them feel better.\nYou always give 3 simple and most appropriate remedies to the user based on their issue.\nThe user issue is mentioned in the below triple backticks:\n```\n{input}\n```\n'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000026DBE909F40>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000026DBE915460>, openai_api_key=SecretStr('**********'), openai_proxy='')),
 'General Physician': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a helpful general physician who always suggests home remedies for any issues.\nYou always suggest 5 most appropriate reme

In [11]:
print(destination_descriptions)

Psychologist: Helps with any psychological issues
General Physician: Helps with any General Physical Health issues



In [15]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router import MultiPromptChain

In [33]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_descriptions)

In [34]:
router_prompt = PromptTemplate(template=router_template, input_variables=[], output_parser=RouterOutputParser())

In [35]:
router_chain = LLMRouterChain.from_llm(llm=chat, prompt=router_prompt)

In [36]:
from langchain.chains import ConversationChain

default_chain = ConversationChain(llm=chat, output_key='text')

In [37]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [38]:
response = chain.run("I could not sleep last night")
print(response)



> Entering new MultiPromptChain chain...
Psychologist: {'input': 'I had trouble sleeping last night'}
> Finished chain.
1. Establish a bedtime routine: Try going to bed and waking up at the same time every day, even on weekends. Create a relaxing bedtime routine to signal to your body that it's time to wind down.

2. Limit screen time before bed: The blue light emitted from screens can interfere with your body's production of melatonin, the hormone that regulates sleep. Try to avoid screens at least an hour before bedtime.

3. Practice relaxation techniques: Deep breathing, meditation, or progressive muscle relaxation can help calm your mind and body before bed. Experiment with different techniques to find what works best for you.


In [5]:
response = chain.run("I couldn't walk properly since I hit my leg to the bed yesterday")
print(response)



> Entering new MultiPromptChain chain...
General Physician: {'input': "I couldn't walk properly since I hit my leg to the bed yesterday"}
> Finished chain.
Here are 5 home remedies for your leg injury:

1. Rest: Make sure to give your leg proper rest to allow it to heal.
2. Ice: Apply ice to the injured area to reduce swelling and pain. Make sure to wrap the ice pack in a cloth to prevent ice burn.
3. Elevation: Elevate your leg above heart level to reduce swelling and promote healing.
4. Compression: Wrap the injured area with a compression bandage to provide support and reduce swelling.
5. Arnica: Consider using arnica gel or cream to help reduce pain and inflammation in the injured area.


In [6]:
response = chain.run("Taj mahal is beautiful")
print(response)



> Entering new MultiPromptChain chain...
None: {'input': 'Taj mahal is beautiful'}
> Finished chain.
Yes, the Taj Mahal is indeed a breathtakingly beautiful structure located in Agra, India. It was built by the Mughal emperor Shah Jahan in memory of his wife Mumtaz Mahal and is considered one of the most iconic symbols of love and architecture in the world. The Taj Mahal is made of white marble and features intricate carvings, inlaid precious stones, and a stunning garden complex. It is also a UNESCO World Heritage Site and attracts millions of visitors each year.


In [9]:
response = chain.run("I don't like taj mahal")
print(response)



> Entering new MultiPromptChain chain...
Psychologist: {'input': "I don't like taj mahal"}
> Finished chain.
I understand that not everyone may appreciate the Taj Mahal for various reasons. Here are three simple remedies that may help you feel better about the situation:

1. Focus on other aspects of travel or landmarks that interest you more. It's okay to have different preferences and explore other destinations that align better with your interests.

2. Try to appreciate the historical and cultural significance of the Taj Mahal. Learning about its architecture, history, and symbolism may help you gain a deeper understanding and appreciation for this iconic monument.

3. Plan a trip to a different destination that you feel more excited about. Traveling is about discovering new places and experiences that resonate with you, so don't hesitate to explore other options that align better with your preferences.


## MathChain

In [8]:
!pip install numexpr

### LLMMathChain

In [10]:
from langchain import LLMMathChain

llm_math_chain = LLMMathChain.from_llm(llm=chat, verbose=True)

In [11]:
llm_math_chain.run("What is 23 multiplied by 4 and subtracted by 18?")



> Entering new LLMMathChain chain...
What is 23 multiplied by 4 and subtracted by 18?```text
23 * 4 - 18
```
...numexpr.evaluate("23 * 4 - 18")...

Answer: 74
> Finished chain.


'Answer: 74'

### PALChain

In [12]:
from langchain_experimental.pal_chain import PALChain

In [13]:
pal_chain = PALChain.from_math_prompt(llm=chat, verbose=True)

In [14]:
pal_chain.run("A train travels at a speed of 60km/h and covers a certain distance in 3 hours. What is the distance covered by the train?")



> Entering new PALChain chain...


Python REPL can execute arbitrary code. Use with caution.


def solution():
    """A train travels at a speed of 60km/h and covers a certain distance in 3 hours. What is the distance covered by the train?"""
    speed = 60  # speed in km/h
    time = 3  # time in hours
    distance_covered = speed * time
    result = distance_covered
    return result

> Finished chain.


'180'

In [ ]:
# Task

from langchain.chains import MultiRetrievalQAChain

chain = MultiRetrievalQAChain.from_retrievers(
    llm=chat,
    retriever_infos, 
    default_chain
)